# Conservative SDOF - Harmonic Balance (3)

- Good for strongly and weakly NL systems
- Requires significant knowledge about system studied 
- Similar to Galerkin method, but in time

## Overview

Assume an harmonic expansion for $x(t)$

$$
\begin{gather*}
     x(t) = \sum^N_{m = 0} A_m \cos\left( m\omega t + m \beta_0\right)
\end{gather*}
$$

substitute this into the EOM and enforce zero projected residual, i.e. taking the inner product with orthogonal assumption.

$$
\begin{gather*}
    \left< EOM_{LHS},\phantom{-} \cos\left(m\omega t + m\beta_0\right) \right> = 0\\
    m = 0, 1, 2, \cdots, N
\end{gather*}
$$

which is 

$$
\begin{gather*}
    \int_0^{\frac{2\pi}{\omega} } \left( EOM_{LHS}\bullet \cos\left(m\omega t + m\beta_0\right) \right)dt
\end{gather*}
$$

Then we solve for $A_0, A_2, A_3, \cdots, A_n$ and $\omega$ in terms of $A_1$ and other system parameters. In this example we solve the following equation

$$
\begin{gather*}
    \ddot{x} + \omega_0^2 x + \alpha_2 x^2 + \alpha_3 x^3 = 0.
\end{gather*}
$$

## Assumed Harmonic Expansion - up to and including $\cos\left(2\left(\omega t + \beta \right)\right)$ and $\sin\left(2\left(\omega t + \beta \right)\right) $ 

### $\sin\phi$ term turns out having a zero coefficient

Assume a harmonic expansion for $x(t)$ - will use this to satisfy the cubic equation, so include harmonic terms at twice the linear natrual frequency 

Note: phi = omega * t + beta0

In [1]:
import sympy as sp
from sympy.simplify.fu import TR4, TR7, TR8, TR11
import numpy as np
from itertools import combinations_with_replacement
from math import factorial

In [37]:
M = 2  # order of harmonic expansion cosine
K = 1  # order of harmonic expansion sine

Ai = sp.symbols('A_(0:' + str(M+K+1) + ')', real=True)  # coefficient A
t, w, w0 = sp.symbols('t omega omega_0', real=True, positive=True)
b0 = sp.Symbol('beta_0')

# harmonic expansion of x(t)
phi = sp.Symbol('phi')
xh = sp.Function('x_h')(phi)
xh = sum([Ai[i] * sp.cos(i * phi) for i in range(M+1)]) + Ai[3] * sp.sin(2 * phi)
xh

A_0 + A_1*cos(phi) + A_2*cos(2*phi) + A_3*sin(2*phi)

In [38]:
# Defining the EOM
N = 3  # order of powered terms of alpha 
ai = sp.symbols('alpha_(2:' + str(N+1) + ')')
x = sp.Function('x')(phi)

# EOM
EOM = w**2 * sp.diff(x, phi, 2) + w0**2 * x + sum([ai[i-2] * x**i for i in range(2, N+1)])
EOM = sp.Eq(EOM, 0)
EOM

Eq(alpha_2*x(phi)**2 + alpha_3*x(phi)**3 + omega**2*Derivative(x(phi), (phi, 2)) + omega_0**2*x(phi), 0)

In [39]:
# Substitute x(t) for the harmonic expansion of x(t) into EOM
err = EOM.lhs.subs(x, xh).doit().expand()
err = TR7(err).expand()
err = TR8(err).expand()
err

A_0**3*alpha_3 + 3*A_0**2*A_1*alpha_3*cos(phi) + 3*A_0**2*A_2*alpha_3*cos(2*phi) + 3*A_0**2*A_3*alpha_3*sin(2*phi) + A_0**2*alpha_2 + 3*A_0*A_1**2*alpha_3*cos(2*phi)/2 + 3*A_0*A_1**2*alpha_3/2 + 3*A_0*A_1*A_2*alpha_3*cos(phi) + 3*A_0*A_1*A_2*alpha_3*cos(3*phi) + 3*A_0*A_1*A_3*alpha_3*sin(phi) + 3*A_0*A_1*A_3*alpha_3*sin(3*phi) + 2*A_0*A_1*alpha_2*cos(phi) + 3*A_0*A_2**2*alpha_3*cos(4*phi)/2 + 3*A_0*A_2**2*alpha_3/2 + 3*A_0*A_2*A_3*alpha_3*sin(4*phi) + 2*A_0*A_2*alpha_2*cos(2*phi) - 3*A_0*A_3**2*alpha_3*cos(4*phi)/2 + 3*A_0*A_3**2*alpha_3/2 + 2*A_0*A_3*alpha_2*sin(2*phi) + A_0*omega_0**2 + 3*A_1**3*alpha_3*cos(phi)/4 + A_1**3*alpha_3*cos(3*phi)/4 + 3*A_1**2*A_2*alpha_3*cos(2*phi)/2 + 3*A_1**2*A_2*alpha_3*cos(4*phi)/4 + 3*A_1**2*A_2*alpha_3/4 + 3*A_1**2*A_3*alpha_3*sin(2*phi)/2 + 3*A_1**2*A_3*alpha_3*sin(4*phi)/4 + A_1**2*alpha_2*cos(2*phi)/2 + A_1**2*alpha_2/2 + 3*A_1*A_2**2*alpha_3*cos(phi)/2 + 3*A_1*A_2**2*alpha_3*cos(3*phi)/4 + 3*A_1*A_2**2*alpha_3*cos(5*phi)/4 + 3*A_1*A_2*A_3*alpha_

Note there are terms with frequency content as high as $6\phi$

To find $A_0$, $\omega$, $A_2$, we project the error back on the assumed expansion and set the projections to zero

Note: same as setting the coefficients in front of each $\cos(n\phi)$ to zero for $n = 0, 1, 2$

In [45]:
proj_eqn = {}
for i in range(M+1):
    proj_eqn['eq'+str(i)] = sp.integrate(sp.expand_trig(err * sp.cos(i * phi)), (phi, 0, 2*sp.pi))

proj_eqn['eq'+str(3)] = sp.integrate(sp.expand_trig(err * sp.sin(2 * phi)), (phi, 0, 2*sp.pi))

In [46]:
proj_eqn['eq0']

2*pi*A_0**3*alpha_3 + 2*pi*A_0**2*alpha_2 + 3*pi*A_0*A_1**2*alpha_3 + 3*pi*A_0*A_2**2*alpha_3 + 3*pi*A_0*A_3**2*alpha_3 + 2*pi*A_0*omega_0**2 + 3*pi*A_1**2*A_2*alpha_3/2 + pi*A_1**2*alpha_2 + pi*A_2**2*alpha_2 + pi*A_3**2*alpha_2

In [47]:
proj_eqn['eq1']

3*pi*A_0**2*A_1*alpha_3 + 3*pi*A_0*A_1*A_2*alpha_3 + 2*pi*A_0*A_1*alpha_2 + 3*pi*A_1**3*alpha_3/4 + 3*pi*A_1*A_2**2*alpha_3/2 + pi*A_1*A_2*alpha_2 + 3*pi*A_1*A_3**2*alpha_3/2 - pi*A_1*omega**2 + pi*A_1*omega_0**2

In [48]:
proj_eqn['eq2']

3*pi*A_0**2*A_2*alpha_3 + 3*pi*A_0*A_1**2*alpha_3/2 + 2*pi*A_0*A_2*alpha_2 + 3*pi*A_1**2*A_2*alpha_3/2 + pi*A_1**2*alpha_2/2 + 3*pi*A_2**3*alpha_3/4 + 3*pi*A_2*A_3**2*alpha_3/4 - 4*pi*A_2*omega**2 + pi*A_2*omega_0**2

In [49]:
proj_eqn['eq3']

3*pi*A_0**2*A_3*alpha_3 + 2*pi*A_0*A_3*alpha_2 + 3*pi*A_1**2*A_3*alpha_3/2 + 3*pi*A_2**2*A_3*alpha_3/4 + 3*pi*A_3**3*alpha_3/4 - 4*pi*A_3*omega**2 + pi*A_3*omega_0**2

if $A_1$ is small neglect terms with $A_0^2, A_2^2, A_0A_2, A_1^2A_0, A_0^3, A_1^2A_2, \cdots$

This is because there is the assumption of 
$$
\begin{gather*}
    A_1 > A_i \phantom{---} i = 0, 2, 3, \cdots
\end{gather*}
$$

equation 0 tells us that $A_0$ is proportional to $A_1$ squared - We have a constant term multiplying $A_0$ (in addition to others) and no $A_1$ terms but only $A_1^2$ terms.

In [50]:
deglist = list(map(int, sp.degree_list(proj_eqn['eq0'], gens=Ai)))
A1_deg = deglist[1]
removing_combos = []
Ai_ = tuple(filter(lambda A: A != Ai[1], Ai))

# A1 ** n -> n can be up to O(A1**4) which is square of A1**2
keep_terms = []
for i in range(A1_deg+1, A1_deg**2+1):
    temp = []
    for j in range(i):
        temp.append(Ai[1])
    keep_terms.append(tuple(temp))

temp = list(combinations_with_replacement(Ai_, A1_deg))
for t in temp:
    if t not in keep_terms:
        removing_combos.append(t)
    
for i in range(A1_deg+1, max(deglist)+1):
    temp = list(combinations_with_replacement(Ai, i))
    for t in temp:
        if t not in keep_terms:
            removing_combos.append(t)
        
eq_copy = proj_eqn['eq0']
for term in removing_combos:
    eq_copy = eq_copy.subs(np.prod(term), 0)

eq_copy = sp.Eq(eq_copy, 0)

eq0_new = eq_copy
eq0_ordered = eq0_new
epsilon = sp.Symbol('epsilon')
for A in Ai_:
    eq0_ordered = eq0_ordered.subs(A, epsilon**A1_deg * A)
eq0_ordered = sp.collect(eq0_ordered.subs(Ai[1], epsilon * Ai[1]).lhs, epsilon)
eq0_ordered = sp.Eq(eq0_ordered, 0)

eq_copy = sp.Eq(Ai[0], sp.solve(eq_copy, Ai[0])[0])
eq_copy

Eq(A_0, -A_1**2*alpha_2/(2*omega_0**2))

equation 2 tells $A_2$ is also proportional to $A_1$ squared 

In [51]:
deglist = list(map(int, sp.degree_list(proj_eqn['eq2'], gens=Ai)))
A1_deg = deglist[1]
Ai_ = tuple(filter(lambda A: A != Ai[1], Ai))

for i in range(A1_deg+1, max(deglist)+1):
    temp = list(combinations_with_replacement(Ai, i))
    for t in temp:
        if t not in removing_combos:
            if t not in keep_terms:
                removing_combos.append(t)

eq_copy = proj_eqn['eq2']
for term in removing_combos:
    eq_copy = eq_copy.subs(np.prod(term), 0)

eq_copy = sp.Eq(eq_copy, 0)

eq2_new = eq_copy
eq2_ordered = eq2_new
for A in Ai_:
    eq2_ordered = eq2_ordered.subs(A, epsilon**A1_deg * A)
eq2_ordered = sp.collect(eq2_ordered.subs(Ai[1], epsilon * Ai[1]).lhs, epsilon)
eq2_ordered = sp.Eq(eq2_ordered, 0)

eq_copy = sp.Eq(Ai[2], sp.solve(eq_copy, Ai[2])[0])
eq_copy

Eq(A_2, A_1**2*alpha_2/(2*(4*omega**2 - omega_0**2)))

equation 3 tells us $A_3$ is also proportional to $A_1^2$

In [53]:
deglist = list(map(int, sp.degree_list(proj_eqn['eq3'], gens=Ai)))
A1_deg = deglist[1]
Ai_ = tuple(filter(lambda A: A != Ai[1], Ai))

for i in range(A1_deg+1, max(deglist)+1):
    temp = list(combinations_with_replacement(Ai, i))
    for t in temp:
        if t not in removing_combos:
            if t not in keep_terms:
                removing_combos.append(t)

eq_copy = proj_eqn['eq3']
for term in removing_combos:
    eq_copy = eq_copy.subs(np.prod(term), 0)

eq_copy = sp.Eq(eq_copy, 0)

eq3_new = eq_copy
eq3_ordered = eq3_new
for A in Ai_:
    eq3_ordered = eq3_ordered.subs(A, epsilon**A1_deg * A)
eq3_ordered = sp.collect(eq3_ordered.subs(Ai[1], epsilon * Ai[1]).lhs, epsilon)
eq3_ordered = sp.Eq(eq3_ordered, 0)

eq_copy = sp.Eq(Ai[3], sp.solve(eq_copy, Ai[3])[0])
eq_copy

Eq(A_3, 0)

In [54]:
eq_copy = proj_eqn['eq1']
for term in removing_combos:
    eq_copy = eq_copy.subs(np.prod(term), 0)

eq_copy = sp.Eq(eq_copy, 0)
eq1_new = eq_copy
eq1_ordered = eq1_new
for A in Ai_:
    eq1_ordered = eq1_ordered.subs(A, epsilon**A1_deg * A)
eq1_ordered = sp.collect(eq1_ordered.subs(Ai[1], epsilon * Ai[1]).lhs, epsilon)
eq1_ordered = sp.Eq(eq1_ordered, 0)

To facilitate the solution, we order the coefficients using epsilon and then ignore all terms quadratic and higher in $A_0$ and $A_2$ - this means we keep terms up to tand including $O(\epsilon^3)$ -> small amplitude assumption

$A_1$ is ordered at $\epsilon$ and $A_0$ and $A_2$ are order at $\epsilon^2$

In [55]:
# Eq0
eq0_ordered

Eq(epsilon**2*(2*pi*A_0*omega_0**2 + pi*A_1**2*alpha_2), 0)

In [56]:
# Eq1 
eq1_ordered

Eq(epsilon**3*(2*pi*A_0*A_1*alpha_2 + 3*pi*A_1**3*alpha_3/4 + pi*A_1*A_2*alpha_2) + epsilon*(-pi*A_1*omega**2 + pi*A_1*omega_0**2), 0)

In [57]:
# Eq2
eq2_ordered

Eq(epsilon**2*(pi*A_1**2*alpha_2/2 - 4*pi*A_2*omega**2 + pi*A_2*omega_0**2), 0)

In [58]:
# Eq 3
eq3_ordered

Eq(epsilon**2*(-4*pi*A_3*omega**2 + pi*A_3*omega_0**2), 0)

In [59]:
A0_sol = sp.solve(eq0_ordered, Ai[0])[0]
A0_sol

-A_1**2*alpha_2/(2*omega_0**2)

In [60]:
A2_sol = sp.solve(eq2_ordered, Ai[2])[0]
A2_sol

A_1**2*alpha_2/(2*(4*omega**2 - omega_0**2))

In [61]:
A3_sol = sp.solve(eq3_ordered, Ai[3])[0]
A3_sol

0

In [62]:
eq1_ordered_update = eq1_ordered.subs([
    (Ai[0], A0_sol), (Ai[2], A2_sol), (Ai[3], A3_sol)
])
omegaSol = sp.solve(eq1_ordered_update, w)

In [63]:
omegaSol[0]

-sqrt(2)*sqrt(-4*A_1**2*alpha_2**2*epsilon**2 + 3*A_1**2*alpha_3*epsilon**2*omega_0**2 + 5*omega_0**4 - sqrt(16*A_1**4*alpha_2**4*epsilon**4 - 24*A_1**4*alpha_2**2*alpha_3*epsilon**4*omega_0**2 + 9*A_1**4*alpha_3**2*epsilon**4*omega_0**4 - 16*A_1**2*alpha_2**2*epsilon**2*omega_0**4 + 18*A_1**2*alpha_3*epsilon**2*omega_0**6 + 9*omega_0**8))/(4*omega_0)

In [64]:
omegaSol[1]

sqrt(2)*sqrt(-4*A_1**2*alpha_2**2*epsilon**2 + 3*A_1**2*alpha_3*epsilon**2*omega_0**2 + 5*omega_0**4 - sqrt(16*A_1**4*alpha_2**4*epsilon**4 - 24*A_1**4*alpha_2**2*alpha_3*epsilon**4*omega_0**2 + 9*A_1**4*alpha_3**2*epsilon**4*omega_0**4 - 16*A_1**2*alpha_2**2*epsilon**2*omega_0**4 + 18*A_1**2*alpha_3*epsilon**2*omega_0**6 + 9*omega_0**8))/(4*omega_0)

In [65]:
omegaSol[2]

-sqrt(2)*sqrt(-4*A_1**2*alpha_2**2*epsilon**2 + 3*A_1**2*alpha_3*epsilon**2*omega_0**2 + 5*omega_0**4 + sqrt(16*A_1**4*alpha_2**4*epsilon**4 - 24*A_1**4*alpha_2**2*alpha_3*epsilon**4*omega_0**2 + 9*A_1**4*alpha_3**2*epsilon**4*omega_0**4 - 16*A_1**2*alpha_2**2*epsilon**2*omega_0**4 + 18*A_1**2*alpha_3*epsilon**2*omega_0**6 + 9*omega_0**8))/(4*omega_0)

In [66]:
omegaSol[3]

sqrt(2)*sqrt(-4*A_1**2*alpha_2**2*epsilon**2 + 3*A_1**2*alpha_3*epsilon**2*omega_0**2 + 5*omega_0**4 + sqrt(16*A_1**4*alpha_2**4*epsilon**4 - 24*A_1**4*alpha_2**2*alpha_3*epsilon**4*omega_0**2 + 9*A_1**4*alpha_3**2*epsilon**4*omega_0**4 - 16*A_1**2*alpha_2**2*epsilon**2*omega_0**4 + 18*A_1**2*alpha_3*epsilon**2*omega_0**6 + 9*omega_0**8))/(4*omega_0)

We are keeping up to $O(\epsilon^4)$ only - use this to get our final expression for $\omega$

In [67]:
# The built in sympy series command takes forever (bad algorithm) so we just implement our original Taylor series function
def taylor(expr, var, upto):
    res = expr.subs(var, 0)
    dexpr = sp.diff(expr, var)
    for n in range(1,upto+1):
        res += dexpr.subs(var, 0) / factorial(n) * var**n
        dexpr = sp.diff(dexpr, var)
    return res + sp.O(var**upto)

In [68]:
omega1 = taylor(omegaSol[0], epsilon, 4)
omega1

-omega_0/2 - epsilon**2*(-4*A_1**2*alpha_2**2 + 3*A_1**2*alpha_3*omega_0**2 - (-16*A_1**2*alpha_2**2*omega_0**4 + 18*A_1**2*alpha_3*omega_0**6)/(6*omega_0**4))/(8*omega_0**3) + O(epsilon**4)

In [69]:
omega2 = taylor(omegaSol[1], epsilon, 4)
omega2

omega_0/2 + epsilon**2*(-4*A_1**2*alpha_2**2 + 3*A_1**2*alpha_3*omega_0**2 - (-16*A_1**2*alpha_2**2*omega_0**4 + 18*A_1**2*alpha_3*omega_0**6)/(6*omega_0**4))/(8*omega_0**3) + O(epsilon**4)

In [70]:
omega3 = taylor(omegaSol[2], epsilon, 4)
omega3

-omega_0 - epsilon**2*(-4*A_1**2*alpha_2**2 + 3*A_1**2*alpha_3*omega_0**2 + (-16*A_1**2*alpha_2**2*omega_0**4 + 18*A_1**2*alpha_3*omega_0**6)/(6*omega_0**4))/(16*omega_0**3) + O(epsilon**4)

In [71]:
omega4 = taylor(omegaSol[3], epsilon, 4)
omega4

omega_0 + epsilon**2*(-4*A_1**2*alpha_2**2 + 3*A_1**2*alpha_3*omega_0**2 + (-16*A_1**2*alpha_2**2*omega_0**4 + 18*A_1**2*alpha_3*omega_0**6)/(6*omega_0**4))/(16*omega_0**3) + O(epsilon**4)

Note: we get a spurious solution - response at $0.5 \omega_0$ !